## Boston University, PY502  10/30/2018

# &nbsp;

# Introduction to the Julia Language

  <img src="julia.png" style="width: 200px;"/>



### Ying-Jer Kao
#### Department of Physics, National Taiwan University
#### yjkao@phys.ntu.edu.tw

## Julia 
#### A dynamic high-level language with the speed of a statically-compiled language

##  Julia is fast!

Julia was designed from the beginning for high performance. Julia programs compile to efficient native code for multiple platforms via LLVM.



![benchmarks](https://julialang.org/images/benchmarks.svg)

<!--- <img src="https://julialang.org/images/benchmarks.svg" width="1200pt"> --->

## Dynamic

Julia is dynamically-typed, feels like a scripting language, and has good support for interactive use.



## Optionally Typed

Julia has a rich language of descriptive datatypes, and type declarations can be used to clarify and solidify programs.


## General

Julia uses multiple dispatch as a paradigm, making it easy to express many object-oriented and functional programming patterns. The standard library provides asynchronous I/O, process control, logging, profiling, a package manager, and more.



## Technical

Julia excels at numerical computing. Its syntax is great for math, many numeric datatypes are supported, and parallelism is available out of the box. Julia's multiple dispatch is a natural fit for defining number and array-like datatypes.



## Composable

Julia packages naturally work well together. Matrices of unit quantities, or data table columns of currencies and colors, just work — and with good performance.


## Key features
   * Just-in-time compiled (using LLVM infrastructure)
   * Dynamic type system
   * Multiple dispatch:
     * define function behavior across many combinations of argument types
     * automatic generation of efficient, specialized code for different argument types
   * Good support for computational science: numerics, statistics, multidimensional arrays, ...
   * Homoiconic and powerful metaprogramming facilities

## Code specialization

In [31]:
function myabs(x)
    if x < 0
        return -x
    end
    return x
end

myabs (generic function with 1 method)

In [32]:
@code_llvm myabs(3) # LLVM code for 64-bit integer


; Function myabs
; Location: In[31]:2
define i64 @julia_myabs_36724(i64) {
top:
; Function <; {
; Location: int.jl:49
  %1 = icmp sgt i64 %0, -1
;}
  br i1 %1, label %L5, label %L3

L3:                                               ; preds = %top
; Location: In[31]:3
; Function -; {
; Location: int.jl:51
  %2 = sub i64 0, %0
;}
  ret i64 %2

L5:                                               ; preds = %top
; Location: In[31]:5
  ret i64 %0
}


## Code specialization

In [33]:
code_llvm(myabs,Tuple{UInt64}) # LLVM code for 64-bit unsigned integer


; Function myabs
; Location: In[31]:2
define i64 @julia_myabs_36725(i64) {
top:
; Location: In[31]:5
  ret i64 %0
}


In [34]:
code_llvm(myabs,Tuple{Float64}) # LLVM code for 64-bit floating point


; Function myabs
; Location: In[31]:2
define double @julia_myabs_36726(double) {
top:
; Function <; {
; Location: float.jl:497
; Function <; {
; Location: float.jl:452
  %1 = fcmp uge double %0, 0.000000e+00
;}}
  br i1 %1, label %L11, label %L9

L9:                                               ; preds = %top
; Location: In[31]:3
; Function -; {
; Location: float.jl:387
  %2 = fsub double -0.000000e+00, %0
;}
  ret double %2

L11:                                              ; preds = %top
; Location: In[31]:5
  ret double %0
}


## Type inference & type stability

In [35]:
mysqrt(x) = x < zero(x) ? sqrt(complex(x)) : sqrt(x)
code_warntype(mysqrt,Tuple{Float64})

Body::Union{Complex{Float64}, Float64}
│╻    <1 1 ─ %1  = (Base.lt_float)(x, 0.0)::Bool
│      └──       goto #3 if not %1
││╻╷╷  Type  2 ─ %3  = %new(Complex{Float64}, x, 0.0)::Complex{Float64}
│      │   %4  = invoke Main.sqrt(%3::Complex{Float64})::Complex{Float64}
│      └──       return %4
││╻    <  3 ─ %6  = (Base.lt_float)(x, 0.0)::Bool
││     └──       goto #5 if not %6
││     4 ─       invoke Base.Math.throw_complex_domainerror(:sqrt::Symbol, _2::Float64)
││     └──       $(Expr(:unreachable))
││     5 ─ %10 = (Base.Math.sqrt_llvm)(x)::Float64
││     └──       goto #6
│      6 ─       return %10


## Type inference & type stability

In [36]:
function summyabs(v::Vector)
    s = myabs(v[1])
    for i = 2:length(v)
        s += abs(v[i])
    end
    return s
end
code_warntype(summyabs, Tuple{Vector{Float64}})

Body::Float64
│╻     getindex2 1 ── %1  = (Base.arrayref)(true, v, 1)::Float64
││╻     <  │    %2  = π (0.0, Float64)
│││╻     <  │    %3  = (Base.lt_float)(%1, %2)::Bool
│││     │    %4  = π (0.0, Float64)
│││╻     ==  │    %5  = (Base.eq_float)(%1, %4)::Bool
│││╻     &  │    %6  = (Base.and_int)(%5, true)::Bool
││││    │    %7  = (Base.and_int)(%6, false)::Bool
│││╻     |  │    %8  = (Base.or_int)(%3, %7)::Bool
││      └───       goto #3 if not %8
││╻     -  2 ── %10 = (Base.neg_float)(%1)::Float64
││      └───       goto #4
││      3 ──       goto #4
│       4 ┄─ %13 = φ (#2 => %10, #3 => %1)::Float64
│╻     length3 │    %14 = (Base.arraylen)(v)::Int64
││╻╷╷╷  Type  │    %15 = (Base.sle_int)(2, %14)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%14, 2)
││││    │    %17 = (Base.ifelse)(%15, %14, 1)::Int64
││╻╷╷   isempty  │    %18 = (Base.slt_int)(%17, 2)::Bool
││      └───       goto #6 if not %18
││      5 ──       goto #7
││      6 ──       goto #7
│       7 ┄─ %22 = φ 

In [37]:
x=randn(100)
@time summyabs(x)
x=randn(1000)
@time summyabs(x)
x=randn(10000)
@time summyabs(x)

  0.008435 seconds (14.82 k allocations: 770.828 KiB)
  0.000002 seconds (5 allocations: 176 bytes)
  0.000014 seconds (5 allocations: 176 bytes)


8082.787690217848

## Type inference & type stability

In [38]:
function summysqrt(v::Vector)
    s = mysqrt(v[1])
    for i = 2:length(v)
        s += mysqrt(v[i])
    end
    return s
end
code_warntype(summysqrt,Tuple{Vector{Int64}})

Body::Union{Complex{Float64}, Float64}
│╻      getindex2 1 ── %1  = (Base.arrayref)(true, v, 1)::Int64
││╻      <  │    %2  = (Base.slt_int)(%1, 0)::Bool
││       └───       goto #3 if not %2
│││╻╷╷╷   float  2 ── %4  = (Base.sitofp)(Float64, %1)::Float64
││││╻╷     float  │    %5  = (Base.sitofp)(Float64, 0)::Float64
│││││╻      Type  │    %6  = %new(Complex{Float64}, %4, %5)::Complex{Float64}
│││      │    %7  = invoke Base.sqrt(%6::Complex{Float64})::Complex{Float64}
││       └───       goto #8
│││╻╷╷    float  3 ── %9  = (Base.sitofp)(Float64, %1)::Float64
││││╻      <  │    %10 = (Base.lt_float)(%9, 0.0)::Bool
││││     └───       goto #5 if not %10
││││     4 ──       invoke Base.Math.throw_complex_domainerror(:sqrt::Symbol, %9::Float64)
││││     └───       $(Expr(:unreachable))
││││     5 ── %14 = (Base.Math.sqrt_llvm)(%9)::Float64
││││     └───       goto #6
│││      6 ──       goto #7
││       7 ──       goto #8
│        8 ┄─ %18 = φ (#2 => %7, #7 => %14)::Union{Complex{Float64

In [39]:
x=randn(100)
@time summysqrt(x)
x=randn(1000)
@time summysqrt(x)
x=randn(10000)
@time summysqrt(x)

  0.061757 seconds (21.67 k allocations: 1.099 MiB)
  0.000045 seconds (2.00 k allocations: 54.859 KiB)
  0.000426 seconds (20.00 k allocations: 546.063 KiB)


4158.404155194087 + 4026.7948290669874im

ErrorException: function rand does not accept keyword arguments

## Homoiconicity (Code as data)

In [44]:
ex=quote
    function summysqrt(v::Vector)
        s = mysqrt(v[1])
        for i = 2:length(v)
            s += mysqrt(v[i])
        end
        return x
        end
end

quote
    #= In[44]:2 =#
    function summysqrt(v::Vector)
        #= In[44]:3 =#
        s = mysqrt(v[1])
        #= In[44]:4 =#
        for i = 2:length(v)
            #= In[44]:5 =#
            s += mysqrt(v[i])
        end
        #= In[44]:7 =#
        return x
    end
end

In [45]:
typeof(ex)

Expr

## Homoiconicity 

In [46]:
Meta.show_sexpr(ex)

(:block,
  :(#= In[44]:2 =#),
  (:function, (:call, :summysqrt, (:(::), :v, :Vector)), (:block,
      :(#= In[44]:3 =#),
      (:(=), :s, (:call, :mysqrt, (:ref, :v, 1))),
      :(#= In[44]:4 =#),
      (:for, (:(=), :i, (:call, :(:), 2, (:call, :length, :v))), (:block,
          :(#= In[44]:5 =#),
          (:+=, :s, (:call, :mysqrt, (:ref, :v, :i)))
        )),
      :(#= In[44]:7 =#),
      (:return, :x)
    ))
)

## Metaprogramming

In [47]:
macro twice(ex)
    Expr(:block, esc(ex), esc(ex))
end

@twice (macro with 1 method)

In [48]:
x=3;
@twice x+=1
x

5

In [49]:
macroexpand(Main, :(@twice x+=1))

quote
    x += 1
    x += 1
end